In [11]:

import time
import numpy as np
import cv2
from matplotlib import pyplot as plt
import math
#from matplotlib.widgets import Slider, Button, RadioButtons
from scipy import signal
from scipy import ndimage
import cmath
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column, row

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import clear_output
focuswidget=widgets.FloatSlider(
    value=0,
    min=-3,
    max=3.0,
    step=0.02,
    description='Focus:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

lsawidget=widgets.FloatSlider(
    value=0,
    min=-1,
    max=1.0,
    step=0.01,
    description='LowerSpherical:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

diameterwidget=widgets.IntSlider(
    value=50,
    min=25,
    max=200,
    step=5,
    description='Diameter(mm):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

output_notebook()

#TODO select image
img = np.float32(cv2.imread('saturn.jpg',0))
#img = np.float32(cv2.imread('saturn.jpg',0))

#store original image max intensity
#img_max=img.max()
cv2.normalize(img,img,0, 1, cv2.NORM_MINMAX)

#dimension of starimage-generator
dimension=512
halfdim=256

#magstar is star intensity
magstar=np.zeros((dimension,dimension), dtype=float)
#realstar is visual star intensity
realstar=np.ones((dimension,dimension),dtype=float)
phi=np.ones((dimension,dimension),dtype=float)
#very slow routine
for x in range(dimension):
    dx=(x-halfdim)/dimension
    for y in range(dimension):
        dy=(y-halfdim)/dimension
        phi=math.atan2(dy,dx)

start = time.time()

def report_dtime(blockname):
    global start
    end = time.time()
    print(blockname)
    print(end - start)
    start=end


def gen_star(val):
    global start 
    global magstar
    start = time.time()
    
    np_real=np.zeros((dimension,dimension), dtype=float)
    np_imag=np.zeros((dimension,dimension),dtype=float)
    
    #radius=10 simulates 50mm scope with dimension 256 for jupiter (300px = 10pix/arcsecond)
    #radius=10 simulates 100mm scope with dimension 128 for jupiter
    #radius=20 simulates 100mm scope with dimension 256 for jupiter
    #diam 25mm is 5 at 256 = diam/25*5
    #diam 25mm is 20 at 1024 = diam/25*20
    #10pix/arcsecond diam/25*(dimension/256)
    diampos=diameterwidget.value
    focuspos=focuswidget.value
    obspos=0
    lsapos=lsawidget.value
    waveradius=diampos/25*(5*dimension/256)
    diam2=waveradius*waveradius*1.0
    divdiam2=1/diam2;
    obstruction=obspos/100
    focus=focuspos*1.0
    lsa=lsapos*4.0
    twopi=math.pi*2.0
    onesixth=1/6
    #count=0
         
    
    #setup radius matrix
    def radi(i, j):
        r2=((i -halfdim)**2 + (j-halfdim)**2 )*divdiam2
        return r2
    radius=np.fromfunction(radi, shape=(dimension, dimension))
    
    #calculate aberrations matrix
    def aberrate(i,j):
        r2=((i -halfdim)**2 + (j-halfdim)**2 )*divdiam2
        v=(radius<=1)*twopi*(r2*focus+ lsa*(r2-r2*r2-onesixth))
        return v
    aberrated=np.fromfunction(aberrate, shape=(dimension, dimension))
    
    #calculate cosine of aberrations matrix
    def aberrate_cos(i, j):
        v= (radius<=1)*(radius>=obstruction)*np.cos(aberrated)
        return v
    cossum=np.fromfunction(aberrate_cos, shape=(dimension, dimension))
    
    #calculate sine of aberrations matrix
    def aberrate_sin(i, j):
        v= (radius<=1)*(radius>=obstruction)*np.sin(aberrated)
        return v
    sinsum=np.fromfunction(aberrate_sin, shape=(dimension, dimension))
        
    #create a plane with two arrays    
    planes = [(cossum), (sinsum)]
    
    complexI = cv2.merge(planes)         # Add to the expanded another plane with zeros
        
    cv2.dft(complexI, complexI)         # this way the result may fit in the source matrix
    cv2.split(complexI, planes)                   # planes[0] = Re(DFT(I), planes[1] = Im(DFT(I))
    
    dft0= np.fft.fftshift(planes[0])
    dft1= np.fft.fftshift(planes[1])
    
    magstar1=cv2.magnitude(dft0,dft1)
    magstar=np.square(magstar1)
        
    cv2.normalize(magstar,magstar,0, 1, cv2.NORM_MINMAX)
    ones=np.ones((dimension,dimension), dtype=magstar.dtype)
    cv2.add(magstar, ones, magstar);
       
    cv2.log(magstar, magstar);
    cv2.normalize(magstar,magstar,0, 1, cv2.NORM_MINMAX)
    
    #ax[1].imshow(magstar, cmap = 'rainbow')
    
    #cv2.normalize(magstar,realstar)          
    psf=magstar[:,256]
    #ax[2].cla()
    #ax[2].plot(psf)
     
    
real=[]
def convolute(val):
    print(convolute)
    gen_star(val)
    
    global real
    blur1 = signal.fftconvolve(img, magstar, mode='full') 
     
    multi=img.max()/blur1.max()
     
    #normalize the maximum intensity back to the original
    real=blur1*multi
    #real[0,0]=bmax*multi     
        
    #add star to convoluted image on scale     
    bmax=magstar.max()*0.2
    multi=img.max()/magstar.max()
                
    for x in range(dimension):
        for y in range(dimension):
           if magstar[x,y]>bmax:
              real[x,y]=magstar[x,y]*multi

p = figure(plot_width=512, plot_height=512, x_range=(0, 512), y_range=(0, 512),  
           match_aspect=True,
           tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")])
p.axis.visible = False
p.border_fill_color = None

p1 = figure(plot_width=512, plot_height=512, x_range=(0, 512), y_range=(0, 512),   
            match_aspect=True,
           tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")])
p1.axis.visible = False
p1.border_fill_color = None

def conv(change):
    global real
    global magstar
    convolute(0)
    p.image(image=[magstar],x=0, y=0, dw=512, dh=512, palette="Greys256")
    p1.image(image=[real],x=0, y=0, dw=512, dh=512, palette="Greys256")
    layout = row(p,p1)
    with out:
        clear_output(wait=True)
        show(layout)
    
lsawidget.observe(conv, names="value")    
diameterwidget.observe(conv, names="value")
focuswidget.observe(conv, names="value")
out = widgets.Output(layout=widgets.Layout(overflow_y='scroll'))
display(widgets.VBox([widgets.HBox([diameterwidget, focuswidget, lsawidget ]),out]))


Loading BokehJS ...